In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
for i in range(1,91):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    data['S'+str(i)] = [
                {
                    'action': np.array([struct[0,0]['resp'].flatten()])-1,
                    'state': np.array([struct[0,0]['stimOffers']]),
                    'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]


In [3]:
data

{'S1': [{'action': array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
           1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
           1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
           1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
           0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
           1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
           0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
           0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
           0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1]], dtype=uint8),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=5)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-01 13:40:13,040 - DeepRL - DEBUG - version control: (None, None)
2021-07-01 13:40:13,041 - DeepRL - DEBUG - learning rate: 0.01
2021-07-01 13:40:13,041 - DeepRL - DEBUG - global iters: 50
2021-07-01 13:40:13,041 - DeepRL - DEBUG - training data-points: 90
2021-07-01 13:40:13,044 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-01 13:40:13,478 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-01 13:40:19,618 - DeepRL - DEBUG - global iter =    0 total obj: 11936.3071
2021-07-01 13:40:25,319 - DeepRL - DEBUG - global iter =    1 total obj: 11901.7473
2021-07-01 13:40:31,464 - DeepRL - DEBUG - global iter =    2 total obj: 11862.7441
2021-07-01 13:40:37,088 - DeepRL - DEBUG - global iter =    3 total obj: 11800.1317
2021-07-01 13:40:42,737 - DeepRL - DEBUG - global iter =    4 total obj: 11744.4089
2021-07-01 13:40:48,420 - DeepRL - DEBUG - global iter =    5 total obj: 11668.0413
2021-07-01 13:40:53,959 - DeepRL - DEBUG - global iter =    6 total obj: 11616.7607
2021-07-01 13:40:59,633 - DeepRL - DEBUG - global iter =    7 total obj: 11619.1595
2021-07-01 13:41:05,318 - DeepRL - DEBUG - global iter =    8 total obj: 11539.8176
2021-07-01 13:41:11,183 - DeepRL - DEBUG - global iter =    9 total obj: 11499.3301
2021-07-01 13:41:17,527 - DeepRL - DEBUG - global iter =   10 total obj: 11430.4020
2021-07-01 13:41:23,836 - DeepRL - DEBUG - global iter =   11 total obj: 113

INFO:tensorflow:../results/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 25,   0,   0, ..., -25,   0,   0],
          [ 25,   0,   0, ..., -25,   0,   0],
          [  0,   0,   0, ..., -25,   0,   0],
          ...,
          [  0,   0,   0, ...,   0, -25, -25],
          [ 25,  25,   0, ...,   0, -25,   0],
          [ 25,   0,   0, ..., -25,   0,   0]], dtype=int16),
   'action': array([[0, 0, 1, ..., 0, 1, 1],
          [0, 0, 1, ..., 0, 1, 1],
          [0, 0, 0, ..., 0, 0, 1],
          ...,
          [0, 0, 1, ..., 1, 0, 1],
          [0, 1, 0, ..., 1, 1, 0],
          [0, 0, 1, ..., 0, 1, 1]], dtype=uint8),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          [[1, 3],
           [2, 1],
           [1, 2],
           ...,
           [1, 2],
           [3, 1],
           [2, 3]],
   
    

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

In [7]:
test_data = {}
for i in range(91,101):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    test_data['S'+str(i)] = [
                {
                    'action': np.array([struct[0,0]['resp'].flatten()])-1,
                    'state': np.array([struct[0,0]['stimOffers']]),
                    'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]


and we want to test the model every 10 iterations:

In [8]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-01 13:45:04,876 - DeepRL - DEBUG - version control: (None, None)
2021-07-01 13:45:04,877 - DeepRL - DEBUG - learning rate: 0.01
2021-07-01 13:45:04,878 - DeepRL - DEBUG - global iters: 50
2021-07-01 13:45:04,879 - DeepRL - DEBUG - training data-points: 90
2021-07-01 13:45:04,880 - DeepRL - DEBUG - test data-points: 10
2021-07-01 13:45:05,287 - DeepRL - DEBUG - opt started...
2021-07-01 13:45:05,287 - DeepRL - DEBUG - started testing...
2021-07-01 13:45:06,796 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-01 13:45:13,396 - DeepRL - DEBUG - global iter =    0 total obj: 11944.3185
2021-07-01 13:45:19,009 - DeepRL - DEBUG - global iter =    1 total obj: 11857.3195
2021-07-01 13:45:24,835 - DeepRL - DEBUG - global iter =    2 total obj: 11761.0793
2021-07-01 13:45:31,220 - DeepRL - DEBUG - global iter =    3 total obj: 11626.9627
2021-07-01 13:45:37,105 - DeepRL - DEBUG - global iter =    4 total obj: 11552.4358
2021-07-01 13:45:42,928 - DeepRL - DEBUG - global iter =    5 total obj: 11495.5492
2021-07-01 13:45:48,660 - DeepRL - DEBUG - global iter =    6 total obj: 11435.6949
2021-07-01 13:45:54,359 - DeepRL - DEBUG - global iter =    7 total obj: 11386.7330
2021-07-01 13:46:00,045 - DeepRL - DEBUG - global iter =    8 total obj: 11364.9352
2021-07-01 13:46:05,872 - DeepRL - DEBUG - global iter =    9 total obj: 11343.5572
2021-07-01 13:46:05,873 - DeepRL - DEBUG - started testing...
2021-07-01 13:46:07,259 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-01 13:46:13,413 - DeepRL - DEBUG - global iter =   10 total obj: 11313.2262
2021-07-01 13:46:19,161 - DeepRL - DEBUG - global iter =   11 total obj: 11299.6564
2021-07-01 13:46:25,109 - DeepRL - DEBUG - global iter =   12 total obj: 11253.3173
2021-07-01 13:46:31,328 - DeepRL - DEBUG - global iter =   13 total obj: 11212.4898
2021-07-01 13:46:37,724 - DeepRL - DEBUG - global iter =   14 total obj: 11156.9481
2021-07-01 13:46:43,507 - DeepRL - DEBUG - global iter =   15 total obj: 11121.9457
2021-07-01 13:46:49,720 - DeepRL - DEBUG - global iter =   16 total obj: 11087.1021
2021-07-01 13:46:55,376 - DeepRL - DEBUG - global iter =   17 total obj: 11084.8974
2021-07-01 13:47:00,876 - DeepRL - DEBUG - global iter =   18 total obj: 11055.2963
2021-07-01 13:47:06,469 - DeepRL - DEBUG - global iter =   19 total obj: 11036.5048
2021-07-01 13:47:06,470 - DeepRL - DEBUG - started testing...
2021-07-01 13:47:07,832 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-01 13:48:12,048 - DeepRL - DEBUG - global iter =   30 total obj: 10777.9338
2021-07-01 13:48:17,874 - DeepRL - DEBUG - global iter =   31 total obj: 10750.1349
2021-07-01 13:48:23,532 - DeepRL - DEBUG - global iter =   32 total obj: 10736.0038
2021-07-01 13:48:29,551 - DeepRL - DEBUG - global iter =   33 total obj: 10731.0717
2021-07-01 13:48:35,307 - DeepRL - DEBUG - global iter =   34 total obj: 10731.4535
2021-07-01 13:48:40,921 - DeepRL - DEBUG - global iter =   35 total obj: 10715.3480
2021-07-01 13:48:46,890 - DeepRL - DEBUG - global iter =   36 total obj: 10704.3525
2021-07-01 13:48:52,665 - DeepRL - DEBUG - global iter =   37 total obj: 10695.0668
2021-07-01 13:48:58,540 - DeepRL - DEBUG - global iter =   38 total obj: 10669.8808
2021-07-01 13:49:04,401 - DeepRL - DEBUG - global iter =   39 total obj: 10649.9371
2021-07-01 13:49:04,402 - DeepRL - DEBUG - started testing...
2021-07-01 13:49:05,932 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-01 13:49:11,882 - DeepRL - DEBUG - global iter =   40 total obj: 10627.9240
2021-07-01 13:49:17,561 - DeepRL - DEBUG - global iter =   41 total obj: 10629.3277
2021-07-01 13:49:23,321 - DeepRL - DEBUG - global iter =   42 total obj: 10616.0637
2021-07-01 13:49:28,910 - DeepRL - DEBUG - global iter =   43 total obj: 10600.3102
2021-07-01 13:49:34,583 - DeepRL - DEBUG - global iter =   44 total obj: 10599.7506
2021-07-01 13:49:40,047 - DeepRL - DEBUG - global iter =   45 total obj: 10563.9698
2021-07-01 13:49:45,782 - DeepRL - DEBUG - global iter =   46 total obj: 10576.1482
2021-07-01 13:49:51,424 - DeepRL - DEBUG - global iter =   47 total obj: 10539.7047
2021-07-01 13:49:56,995 - DeepRL - DEBUG - global iter =   48 total obj: 10521.5873
2021-07-01 13:50:02,512 - DeepRL - DEBUG - global iter =   49 total obj: 10537.1755
2021-07-01 13:50:02,513 - DeepRL - DEBUG - opt finished.
2021-07-01 13:50:02,514 - DeepRL - DEBUG - started testing...
2021-07-01 13:50:03,899 - DeepRL - DEBUG 

INFO:tensorflow:../results/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
test_data["S100"]

[{'action': array([[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
          0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
          1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
          1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
          1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
          0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
          1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
          0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1]], dtype=uint8),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [